In [1]:
!pip install pyspark

In [82]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]").getOrCreate()
sc = spark.sparkContext
#from google.colab import drive
#drive.mount('/My_drive/')

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46679)
Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46679)

In [3]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [7]:
#Charger un csv
df = spark.read.csv("./vgsales.csv",header=True)
#Compter le nombre de lignes
print(df.count())
#Afficher les 20 premières lignes
df.show()

16598
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|          33|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10

In [8]:
df.dtypes

[('Rank', 'string'),
 ('Name', 'string'),
 ('Platform', 'string'),
 ('Year', 'string'),
 ('Genre', 'string'),
 ('Publisher', 'string'),
 ('NA_Sales', 'string'),
 ('EU_Sales', 'string'),
 ('JP_Sales', 'string'),
 ('Other_Sales', 'string'),
 ('Global_Sales', 'string')]

## Caster vos colonnes

In [9]:
df = df.withColumn("NA_Sales",df["NA_Sales"].cast("double"))
df = df.withColumn("EU_Sales",df["EU_Sales"].cast("double"))
df = df.withColumn("JP_Sales",df["JP_Sales"].cast("double"))
df = df.withColumn("Other_Sales",df["Other_Sales"].cast("double"))
df = df.withColumn("Global_Sales",df["Global_Sales"].cast("double"))
df = df.withColumn("Year",df["Year"].cast("Int"))

df.dtypes

[('Rank', 'string'),
 ('Name', 'string'),
 ('Platform', 'string'),
 ('Year', 'int'),
 ('Genre', 'string'),
 ('Publisher', 'string'),
 ('NA_Sales', 'double'),
 ('EU_Sales', 'double'),
 ('JP_Sales', 'double'),
 ('Other_Sales', 'double'),
 ('Global_Sales', 'double')]

<h1> Compter le nombre de jv Nintendo </h1>

In [10]:
#Via un DF
df.where(df.Publisher == 'Nintendo').count()
#df.where("Publisher == 'Nintendo'").count()

703

In [11]:
#Via un RDD
#df.rdd.filter(....
#Voir corriger

In [12]:
#Via du SQL
df.createOrReplaceTempView("jv_vente")
requete_sql = spark.sql("select count(*) from jv_vente where Publisher = 'Nintendo'")
requete_sql.show()

+--------+
|count(1)|
+--------+
|     703|
+--------+



<h1>Rajouter une colonne concernant le pourcentage de vente EU </h1>

In [19]:
#Via un Dataframe

def pourcentage_calcul(a,b):
    return float(a)/float(b)*100

df_EUpercent = df.withColumn('pourcent_EU', df.EU_Sales/df.Global_Sales*100)
df_EUpercent.select('Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'EU_Sales', 'Global_Sales', 'pourcent_EU').show()

+----+--------------------+--------+----+------------+--------------------+--------+------------+------------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|EU_Sales|Global_Sales|       pourcent_EU|
+----+--------------------+--------+----+------------+--------------------+--------+------------+------------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   29.02|       82.74| 35.07372492144066|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|    3.58|       40.24| 8.896620278330019|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   12.88|       35.82| 35.95756560580681|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   11.01|        33.0| 33.36363636363636|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|    8.89|       31.37|28.339177558176605|
|   6|              Tetris|      GB|1989|      Puzzle|          

In [36]:
# DF méthode numéro 2 (UDF, complexe)
def pourcentage_calcul(a,b):
    return float(a)/float(b)*100

df_EUpercent = df.withColumn('pourcent_EU', F.udf( lambda a,b : pourcentage_calcul(a,b), T.FloatType())("EU_Sales","Global_Sales"))
df_EUpercent.show()
#df_EUpercent.select('Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'EU_Sales', 'Global_Sales','pourcent_EU').show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+-----------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|pourcent_EU|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+-----------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|  35.073727|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|   8.896621|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|  35.957565|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|  33.363636|
|   5|Pokemon Red/Po

Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [ ]:
#Via un RDD


In [22]:
#Via du SQL
requete_pourcentEU = spark.sql("SELECT Rank, Name, Platform, Year, Genre, Publisher, EU_Sales, Global_Sales, EU_Sales/Global_Sales*100 AS pourcent_EU FROM jv_vente")
requete_pourcentEU.show()

+----+--------------------+--------+----+------------+--------------------+--------+------------+------------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|EU_Sales|Global_Sales|       pourcent_EU|
+----+--------------------+--------+----+------------+--------------------+--------+------------+------------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   29.02|       82.74| 35.07372492144066|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|    3.58|       40.24| 8.896620278330019|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   12.88|       35.82| 35.95756560580681|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   11.01|        33.0| 33.36363636363636|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|    8.89|       31.37|28.339177558176605|
|   6|              Tetris|      GB|1989|      Puzzle|          

<h1> Afficher les différentes plateformes du dataframe </h1>

In [42]:
df.select('Platform').drop_duplicates().show()

+--------+
|Platform|
+--------+
|     3DO|
|      PC|
|     PS3|
|     NES|
|      PS|
|      DC|
|     GEN|
|     PS2|
|     3DS|
|    PCFX|
|      GG|
|    WiiU|
|    SNES|
|      GB|
|     SCD|
|     N64|
|     PS4|
|     PSP|
|    2600|
|    XOne|
+--------+
only showing top 20 rows



<h1> Afficher les ventes des jeux publiés en 2006 </h1>

In [37]:
df.where(df.Year == 2006).show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   7|New Super Mario B...|      DS|2006|    Platform|            Nintendo|   11.38|    9.23|     6.5|        2.9|       30.01|
|   8|            Wii Play|     Wii|2006|        Misc|            Nintendo|   14.03|     9.2|    2.93|       2.85|       29.02|
|  21|Pokemon Diamond/P...|      DS|2006|Role-Playing|            Nintendo|    6.42|    4.52|    6.04|       1.37|       18.36|
| 101|The Legend of Zel...|     Wii|2006|      Action|            Nintendo|    3.83|    2.19|     0.6|  

<h1>Combien de jeux ont été vendus en plus 15 millions d'exemplaire </h1>

In [38]:
df.where(df.Global_Sales >= 15).count()

28

<h1>Rajouter une colonne pour indiquer si le jeux est un jeux pokémon (2 façons différentes)</h1>

In [57]:
df_isPokemon = df.withColumn('isPokemon', F.lower(df.Name).contains('pokemon')).show()
df_isPokemon.where(df_isPokemon.isPokemon == 'true').count()

35

### Afficher la répartition des mots

In [43]:
#methode par RDD
df.rdd.filter(lambda x : len(x.Name)>0)\
    .flatMap(lambda x :  x.Name.lower().split())\
    .map(lambda x : (x,1))\
    .reduceByKey(lambda a,b :a+b)\
    .toDF(["word","count"]).show()

+------------+-----+
|        word|count|
+------------+-----+
|         wii|   43|
|      sports|   82|
|       super|  289|
|       mario|  110|
|       bros.|   24|
|        kart|   20|
|      resort|   12|
|     pokemon|   35|
| red/pokemon|    1|
|        blue|   33|
|      tetris|   28|
|         new|   76|
|        play|   29|
|        duck|    8|
|        hunt|   17|
|  nintendogs|    3|
|          ds|  166|
|gold/pokemon|    1|
|      silver|   12|
|         fit|   11|
+------------+-----+
only showing top 20 rows



In [50]:
#methode par dataframe
df_1 = df.withColumn("words",F.udf(lambda x :x.lower().split(),\
                                   T.ArrayType(T.StringType()))("Name"))
print(df_1.first())
df_2 =df_1.withColumn("word",F.explode("words"))
print(df_2.first())
df_3 = df_2.groupBy("word").count()
df_3.show()

Row(Rank='1', Name='Wii Sports', Platform='Wii', Year=2006, Genre='Sports', Publisher='Nintendo', NA_Sales=41.49, EU_Sales=29.02, JP_Sales=3.77, Other_Sales=8.46, Global_Sales=82.74, words=['wii', 'sports'])
Row(Rank='1', Name='Wii Sports', Platform='Wii', Year=2006, Genre='Sports', Publisher='Nintendo', NA_Sales=41.49, EU_Sales=29.02, JP_Sales=3.77, Other_Sales=8.46, Global_Sales=82.74, words=['wii', 'sports'], word='wii')


Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


+-----------+-----+
|       word|count|
+-----------+-----+
|battlefront|   10|
|         07|   41|
|        art|   18|
|      urbz:|    6|
|      yoshi|    5|
|     online|   28|
|    icarus:|    1|
|      high:|    7|
|       ac!d|    2|
|     starfy|    1|
|       hope|    3|
|      2000:|    2|
|     travel|    5|
| strawberry|    3|
|      chibi|    3|
|       tmnt|    4|
|    lights,|    4|
|     voyage|    5|
|      kartz|    9|
|     arubai|    1|
+-----------+-----+
only showing top 20 rows



<h1> Quels sont les 20 mots les plus présent </h1>

In [53]:
df_3.sort('count', ascending=False).show()

+------+-----+
|  word|count|
+------+-----+
|   the| 2733|
|    of| 1718|
|     2|  844|
|    no|  743|
|     3|  399|
| world|  386|
|     &|  353|
|    2:|  323|
|   pro|  320|
|  game|  300|
|    to|  289|
| super|  289|
|   and|  260|
|     -|  248|
|  star|  235|
|soccer|  227|
|dragon|  214|
|    ii|  206|
|    in|  198|
|   nba|  193|
+------+-----+
only showing top 20 rows



<h1> Quel mot fait le plus vendre ? </h1>

In [80]:
df_1 = df.withColumn("words",F.udf(lambda x :x.lower().split(),\
                                   T.ArrayType(T.StringType()))("Name"))
print(df_1.first())
df_2 =df_1.withColumn("word",F.explode("words"))
print(df_2.first())
df_3 = df_2.groupBy("word").count()
#df_3 --> word  count
df_4 = df_2.groupBy("word").agg(F.sum("Global_Sales"))
#df_4 --> word  sum(Global_Sales)
df_4.where("word != 'the' and length(word) > 2").sort('sum(Global_Sales)', ascending=False).show()
#df_3.join(df_2)
#df_3.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46679)
Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/m/m/mmoussai/Documents/tp_spark/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46679)

<h1>Afficher les ventes des différents "Publisher" </h1>

<h1> Partie Machine-learning </h1>

<h1> A partir des différentes colonnes (sans les ventes globales) essayez de prédire les ventes européennes </h1>

<h1> Evaluez votre algorithme et essayez en d'autres </h1>

<h1> Rajouter des colonnes pour améliorer vos algorithmes </h1>